In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.csv
/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.hdf
/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.hdf
/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.csv
/kaggle/input/covertype-target/target.csv


In [2]:
from IPython.display import display

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import accuracy_score

from tqdm import tqdm_notebook

from collections import OrderedDict

In [3]:
%%time
# train_df = pd.read_csv('/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.csv')
# test_df = pd.read_csv('/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.csv')
# submission_df = pd.read_csv('/kaggle/input/forest-cover-type-kernels-only/sample_submission.csv')

train_df = pd.read_hdf('/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.hdf')
test_df = pd.read_hdf('/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.hdf')

CPU times: user 68 ms, sys: 908 ms, total: 976 ms
Wall time: 1.47 s


In [4]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,...,0,0,0,0,0,0,0,0,0,5
1,2,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,...,0,0,0,0,0,0,0,0,0,5
2,3,0.130912,-0.160577,-0.887379,0.194243,0.227369,1.106379,0.696843,0.834797,-0.002005,...,0,0,0,0,0,0,0,0,0,2
3,4,0.085421,-0.015231,0.177250,0.070474,1.092853,1.038455,0.827731,0.834797,-0.285268,...,0,0,0,0,0,0,0,0,0,2
4,5,-0.369489,-1.014485,-1.715424,-0.353198,-0.850404,-0.998491,0.238732,0.659368,0.324838,...,0,0,0,0,0,0,0,0,0,5


In [5]:
test_df['Cover_Type'] = np.nan
data_df = pd.concat([train_df, test_df]).drop(columns=['Cover_Type'])

In [6]:
print(train_df.shape)
print(test_df.shape)
print(data_df.shape)

(15120, 56)
(565892, 56)
(581012, 55)


# Read Target Dataset

In [7]:
%%time
target_all = pd.read_csv('/kaggle/input/covertype-target/target.csv',
                        index_col='Id')

y_test = target_all.loc[test_df['Id']]

CPU times: user 228 ms, sys: 20 ms, total: 248 ms
Wall time: 225 ms


# Convert One-Hot encoding to label

In [8]:

# cat_soil = [col for col in data_df.columns if 'Soil' in col]
# data_df['Soil_Type'] = data_df[cat_soil].values.argmax(axis=1)+1
# data_df = data_df.drop(columns=cat_soil)


# cat_wilderness = [col for col in data_df.columns if 'Wilderness' in col]
# data_df['Wilderness_Area'] = data_df[cat_wilderness].values.argmax(axis=1)+1
# data_df = data_df.drop(columns=cat_wilderness)

In [9]:
# data_df.shape

In [10]:
# data_df.columns

In [11]:
# train_df.columns

In [12]:
import featuretools as ft
import featuretools.variable_types as vtypes
from multiprocessing import cpu_count

# Feature Engineering

In [13]:
variable_type = {col: vtypes.Categorical for col in data_df.columns if 'Soil' in col or 'Wilderness' in col}

# variable_type['Cover_Type'] = vtypes.Categorical

# print(variable_type)

In [14]:
seed=47
pipelines = {}

pipelines['Random Forest'] = Pipeline(steps=[('RF', 
                                            RandomForestClassifier(n_jobs=-1,
                                                                   n_estimators=300,
#                                                                    max_features=7,
#                                                                    max_leaf_nodes=220,
                                                                   random_state=seed
                                                                   )
                                                    ),])

pipelines['Extra Trees'] = Pipeline(
                                    steps=[('random forest extra', ExtraTreesClassifier(n_estimators=100, random_state=seed))]
                                    )


pipelines['LGBM'] = Pipeline(steps=[('LightGBM', 
                                    lgb.LGBMClassifier(n_estimators=300,
                                                             num_class=8,
                                                            num_leaves=32,
                                                            random_state=seed,)
                                   ),])


# pipelines['KNN'] = Pipeline(steps=[
#                                         ('KNN', KNeighborsClassifier(n_neighbors=2))
#                                     ])

# Feature Selection Framework
- inputs: 
    1. pipelines for training
    1. pandas.DataFrame
    1. dict of trans_primitive
    1. max_top_mean_num
    1. max_top_var_num
- outputs:
    - ordered dict of features upon trans_primitive

In [15]:
from featuretools.primitives import make_trans_primitive
from featuretools.variable_types import Numeric
from collections import OrderedDict

dict_trans_primitive = OrderedDict()


def squared_root(col1, col2):
    return (col1**2 + col2**2)**0.5

dict_trans_primitive['Squared_Root'] = make_trans_primitive(function=squared_root,
                                                           input_types=[Numeric, Numeric],
                                                           return_type=Numeric,
                                                            commutative=True, # True: combination, False: permutaion
                                                           )


def abs_diff(col1, col2):
    return abs(col1 - col2)

dict_trans_primitive['Abs_Diff'] = make_trans_primitive(function=abs_diff,
                                                       input_types=[Numeric, Numeric],
                                                       return_type=Numeric,
                                                       commutative=True, # True: combination, False: permutaion
                                                       )


def mean_2_cols(col1, col2):
    return (col1.add(col2)) / 2


dict_trans_primitive['Mean_2_Cols'] = make_trans_primitive(function=mean_2_cols,
                                                          input_types=[Numeric, Numeric],
                                                          return_type=Numeric,
                                                          commutative=True, # True: combination, False: permutaion
                                                          )


def mean_3_cols(col1, col2, col3):
    return (col1.add(col2.add(col3))) / 3

dict_trans_primitive['Mean_3_Cols'] = make_trans_primitive(function=mean_3_cols,
                                                          input_types=[Numeric, Numeric, Numeric],
                                                          return_type=Numeric,
                                                          commutative=True, # True: combination, False: permutaion
                                                          )


def div_2_cols(col1, col2):
    eps = 10e-2
    return (col1+eps) / (col2+eps)


dict_trans_primitive['Div_2_Cols'] = make_trans_primitive(function=div_2_cols,
                                                          input_types=[Numeric, Numeric],
                                                          return_type=Numeric,
                                                          commutative=True, # True: combination, False: permutaion
                                                          )

dict_trans_primitive['add_numeric'] = 'add_numeric'
dict_trans_primitive['subtract_numeric'] = 'subtract_numeric'
dict_trans_primitive['multiply_numeric'] = 'multiply_numeric'
# # dict_trans_primitive['divide_numeric'] = 'divide_numeric' # divide by 0.

In [16]:
# %%time
import featuretools as ft
from featuretools.feature_base.feature_base import TransformFeature

from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from tqdm import tqdm, tqdm_notebook
import warnings
import pdb

class FeatureSelection():
    def __init__(self, pipelines,
#                         dict_trans_primitive,
                        trans_primitives,
                        max_top_mean_num,
                        max_top_var_num):
        
        self.pipelines = pipelines
#         self.dict_trans_primitive = dict_trans_primitive
        self.trans_primitives = trans_primitives
        self.max_top_mean_num = max_top_mean_num
        self.max_top_var_num = max_top_var_num
    
    
    def generate_features(self, dataframe,
                         entity_set_name='Dataset',
                         entity_name='forest_table',
                         index_name='Id',
                         max_depth=1):
        
        # define categorical data type
        variable_types = {col: vtypes.Categorical for col in data_df.columns 
                                 if 'Soil' in col or 'Wilderness' in col}
        
        # create database and 1 table
        es = ft.EntitySet(entity_set_name)
        es.entity_from_dataframe(entity_id=entity_name,
                                index=index_name,
                                dataframe=dataframe,
                                variable_types=variable_types)
        
        
#         trans_primitives = [trans_p for name, trans_p in self.dict_trans_primitive.items()]
        
        feature_matrix, features = ft.dfs(entityset=es,
                                         target_entity=entity_name,
                                         trans_primitives=self.trans_primitives,
                                         max_depth=max_depth)

        # To avoid issue, reset index
        feature_matrix = feature_matrix.reset_index()
        
        
        return feature_matrix, features
    
    
    def fit(self, model, X_train, y_train, 
                        X_test, y_test):
    
#         pdb.set_trace()
        
        model.fit(X_train, y_train)

        predictions = model.predict(X_test)
        score = accuracy_score(predictions, y_test)

        return score
    

    def fit_all(self, X_train, y_train,
                      X_test, y_test):
        
        scores = OrderedDict()
        
        for name, p in tqdm_notebook(self.pipelines.items()):
#             print('\rFitting using {:20s}'.format(name), end='')
            print('\t Fitting using {:20s}'.format(name))

            scores[name] = self.fit(p, X_train, y_train,
                                        X_test, y_test)
            
        # erase the printing line
#         print('\r                  {:20s}'.format(' '))
        
        return scores
      
    
    def get_importance_matrix(self, X_train, pipelines):
        
        importances = pd.DataFrame(index=X_train.columns)
        
        for name, p in pipelines.items():
            if 'KNN'.lower() in name.lower():
                pass
            
#             step_name = p.steps[-1][0]
            clf = p.steps[-1][1]
            model = SelectFromModel(clf, prefit=True)
        
            X_train_new = model.transform(X_train)
            X_train_inversed = model.inverse_transform(X_train_new)
            
            X_train_inversed_df = pd.DataFrame(X_train_inversed,
                                                index=X_train.index,
                                                columns=X_train.columns)
            
            importances[name] = X_train_inversed_df.var()
        
        return importances
    
        
    def select_features(self, importance):
        '''
        Pick features in 'top-10 mean list' or 'top-10 variance list'
        
        Args:
            importance (pandas.DataFrame): index=feature, columns=['add_numeric', ... etc]
        '''
        method_cols = importance.columns
        importance['mean'] = importance[method_cols].mean(axis=1)
        importance['var'] = importance[method_cols].var(axis=1)
        
        top_mean = importance['mean'].sort_values(ascending=False)[:self.max_top_mean_num]
        top_var = importance['var'].sort_values(ascending=False)[:self.max_top_var_num]
        
        # get the union of mean and variance
        selected_features = set(top_mean.index).union(set(top_var.index))
        
        return selected_features
        
        
    def generate_and_select_features(self, dataframe, 
                                         indices_train, y_train,
                                         indices_test, y_test):
        
        score_dict = OrderedDict()
        importance_dict = OrderedDict()
        
        '''
        Phase 1: generate the combination of features and get their scores.
        '''
        print('Starting phase 1...')
        # generate features upon self.dict_trans_primitive
        feature_matrix, features = self.generate_features(dataframe)
        feature_matrix = feature_matrix.set_index('Id')
        
        print('\t Shape of original dataframe: {}'.format(dataframe.shape))
        print('\t Shape of feature_matrix:     {}'.format(feature_matrix.shape))
        
        # fit all features
        X_train = feature_matrix.loc[indices_train]
        X_test = feature_matrix.loc[indices_test]
        
#         pdb.set_trace()
        
        score_dict['scores_all'] = self.fit_all(X_train, y_train,
                                            X_test, y_test)
        
#         self.print_importance(X_train, pipelines)
        
        '''
        Phase 2: get scores of selected features.
        '''
        print('Starting phase 2...')
        # generate importance matrix upon different models
        importances = self.get_importance_matrix(X_train=X_train,
                                                 pipelines=self.pipelines)
        
        importance_dict['all'] = importances
        
        # select features except original features
        transform_features = [f.get_name() for f in features if isinstance(f, TransformFeature)]
        
        selected_features = self.select_features(importances.loc[transform_features])
        
        # fit selected features and original features
        combined_features = list(X_train.columns) + list(selected_features)
        
        X_train = feature_matrix.loc[indices_train, combined_features]
        X_test = feature_matrix.loc[indices_test, combined_features]
        
        score_dict['scores_selected'] = self.fit_all(X_train, y_train,
                                                      X_test, y_test)
        
        importances = self.get_importance_matrix(X_train=X_train,
                                                 pipelines=self.pipelines)
        
        importance_dict['selected'] = importances
#         self.print_importance(X_train, pipelines)
        
        
        # copy feature objects
        selected_feature_objs = []
        
        for f in features:
            if f.get_name() in list(selected_features):
                selected_feature_objs.append(f)
                
#         print('Done')
        
        return selected_feature_objs, score_dict, importance_dict
    
    
    def print_importance(self, X_train, pipelines):
        
        importance_matrix = self.get_importance_matrix(X_train,
                                                       pipelines)
        
        if display is not None:
            display(importance_matrix)
        else:
            print(importance_matrix)
            
            

# trans_primitives = [trans_p for name, trans_p in dict_trans_primitive.items()]

# featureSelection = FeatureSelection(pipelines=pipelines,
# #                                     dict_trans_primitive=dict_trans_primitive,
#                                     trans_primitives=trans_primitives,
#                                     max_top_mean_num=12,
#                                     max_top_var_num=10)



# indices_train = train_df['Id']
# y_train = train_df['Cover_Type'].astype('category')
# indices_test = test_df['Id']
    
# selected_feature_objs, scores_all, scores_selected = featureSelection.generate_and_select_features(data_df, 
#                                                                                          indices_train, y_train,
#                                                                                          indices_test, y_test)
    

# Iterate all transform primitives

In [17]:

seed=47

logs = OrderedDict()

indices_train = train_df['Id']
y_train = train_df['Cover_Type'].astype('category')
indices_test = test_df['Id']

for name, trans_primitives in tqdm_notebook(dict_trans_primitive.items()):
    print('Start using transform primitive: {:10s}'.format(name))
    
#     print(trans_primitives)
    if not isinstance(trans_primitives, list):
        trans_primitives = [trans_primitives]
    
    log = {}
    
    # Define pipelines
    pipelines = {}
    pipelines['Random Forest'] = Pipeline(steps=[('RF', 
                                                RandomForestClassifier(n_jobs=-1,
                                                                       n_estimators=300,
    #                                                                    max_features=7,
    #                                                                    max_leaf_nodes=220,
                                                                       random_state=seed
                                                                       )
                                                        ),])

    pipelines['Extra Trees'] = Pipeline(
                                        steps=[('random forest extra', ExtraTreesClassifier(n_estimators=100, random_state=seed))]
                                        )


    # pipelines['LGBM'] = Pipeline(steps=[('LightGBM', 
    #                                     lgb.LGBMClassifier(n_estimators=300,
    #                                                              num_class=8,
    #                                                             num_leaves=32,
    #                                                             random_state=seed,)
    #                                    ),])


    # pipelines['KNN'] = Pipeline(steps=[
    #                                         ('KNN', KNeighborsClassifier(n_neighbors=2))
    #                                     ])

    
    featureSelection = FeatureSelection(pipelines=pipelines,
                                    trans_primitives=trans_primitives,
                                    max_top_mean_num=12,
                                    max_top_var_num=10)
    
    selected_feature_objs, score_dict, importance_dict = featureSelection.generate_and_select_features(data_df, 
                                                                                                         indices_train, y_train,
                                                                                                         indices_test, y_test)
    
    log['feature_objs'] = selected_feature_objs
    log['score_original'] = score_dict['scores_all']
    log['score_selected'] = score_dict['scores_selected']
    log['importance_all'] = importance_dict['all']
    log['importance_selected'] = importance_dict['selected']
    
    logs[name] = log
    
    print()

Start using transform primitive: Squared_Root
Starting phase 1...
	 Shape of original dataframe: (581012, 55)
	 Shape of feature_matrix:     (581012, 99)


	 Fitting using Random Forest       
	 Fitting using Extra Trees         

Starting phase 2...


	 Fitting using Random Forest       
	 Fitting using Extra Trees         


Start using transform primitive: Abs_Diff  
Starting phase 1...
	 Shape of original dataframe: (581012, 55)
	 Shape of feature_matrix:     (581012, 99)


	 Fitting using Random Forest       
	 Fitting using Extra Trees         

Starting phase 2...


	 Fitting using Random Forest       
	 Fitting using Extra Trees         


Start using transform primitive: Mean_2_Cols
Starting phase 1...
	 Shape of original dataframe: (581012, 55)
	 Shape of feature_matrix:     (581012, 99)


...

	 Fitting using Random Forest       
	 Fitting using Extra Trees         


Start using transform primitive: subtract_numeric
Starting phase 1...
	 Shape of original dataframe: (581012, 55)
	 Shape of feature_matrix:     (581012, 99)


	 Fitting using Random Forest       
	 Fitting using Extra Trees         

Starting phase 2...


	 Fitting using Random Forest       
	 Fitting using Extra Trees         


Start using transform primitive: multiply_numeric
Starting phase 1...
	 Shape of original dataframe: (581012, 55)
	 Shape of feature_matrix:     (581012, 99)


	 Fitting using Random Forest       
	 Fitting using Extra Trees         

Starting phase 2...


	 Fitting using Random Forest       
	 Fitting using Extra Trees         





### Print top mean/variance

In [18]:
pd.options.display.max_colwidth = 100

def print_top_mean_importance(importance, num=10):
    display(pd.DataFrame(importance.mean(axis=1).sort_values(ascending=False))[:num])
    
    
def print_top_var_importance(importance, num=10):
    display(pd.DataFrame(importance.var(axis=1).sort_values(ascending=False))[:num])

In [19]:
for name, log in logs.items():
    print('{}:'.format(name))
#     print('importance_all')
    print_top_mean_importance(log['importance_all'])
#     print('importance_selected')
#     print_top_mean_importance(log['importance_selected'])

    print_top_var_importance(log['importance_all'])
    print()

Squared_Root:


,0
Horizontal_Distance_To_Hydrology,1.000066
Horizontal_Distance_To_Roadways,1.000066
Hillshade_9am,1.000066
Elevation,1.000066
Horizontal_Distance_To_Fire_Points,1.000066
Aspect,1.000066
Vertical_Distance_To_Hydrology,1.000066
"SQUARED_ROOT(Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology)",0.663431
"SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",0.624838
"SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)",0.589959


,0
Hillshade_3pm,0.500066
"SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Vertical_Distance_To_Hydrology)",0.173102
"SQUARED_ROOT(Hillshade_9am, Horizontal_Distance_To_Fire_Points)",0.148924
"SQUARED_ROOT(Hillshade_Noon, Horizontal_Distance_To_Roadways)",0.131556
"SQUARED_ROOT(Aspect, Hillshade_9am)",0.064150
Soil_Type4,0.001386
Soil_Type38,0.001050
Soil_Type39,0.000864
"SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",0.000000
Soil_Type18,0.000000



Abs_Diff:


,0
Horizontal_Distance_To_Hydrology,1.000066
Horizontal_Distance_To_Roadways,1.000066
Hillshade_9am,1.000066
Elevation,1.000066
Horizontal_Distance_To_Fire_Points,1.000066
Hillshade_3pm,1.000066
Hillshade_Noon,1.000066
Aspect,1.000066
Vertical_Distance_To_Hydrology,1.000066
"ABS_DIFF(Horizontal_Distance_To_Fire_Points, Vertical_Distance_To_Hydrology)",0.933320


...

,0
Elevation - Slope,2.625453
Horizontal_Distance_To_Roadways - Slope,2.554266
Horizontal_Distance_To_Fire_Points - Slope,2.479218
Hillshade_9am - Vertical_Distance_To_Hydrology,2.192005
Aspect - Horizontal_Distance_To_Fire_Points,2.104478
Hillshade_9am - Horizontal_Distance_To_Hydrology,2.067742
Horizontal_Distance_To_Fire_Points - Vertical_Distance_To_Hydrology,2.030231
Aspect - Elevation,2.022326
Hillshade_9am - Horizontal_Distance_To_Roadways,2.006929
Horizontal_Distance_To_Roadways - Vertical_Distance_To_Hydrology,1.977021


,0
Hillshade_9am,0.500066
Wilderness_Area3,0.029671
Soil_Type3,0.001775
Soil_Type38,0.001050
Soil_Type39,0.000864
Soil_Type23,0.000000
Soil_Type15,0.000000
Soil_Type16,0.000000
Soil_Type17,0.000000
Soil_Type18,0.000000



multiply_numeric:


,0
Horizontal_Distance_To_Hydrology * Vertical_Distance_To_Hydrology,3.118396
Horizontal_Distance_To_Fire_Points * Horizontal_Distance_To_Roadways,1.910398
Elevation * Vertical_Distance_To_Hydrology,1.442414
Horizontal_Distance_To_Hydrology * Horizontal_Distance_To_Roadways,1.402582
Elevation * Horizontal_Distance_To_Hydrology,1.390938
Horizontal_Distance_To_Roadways * Vertical_Distance_To_Hydrology,1.242760
Elevation * Hillshade_9am,1.143018
Aspect * Elevation,1.084430
Elevation * Hillshade_3pm,1.061337
Aspect * Horizontal_Distance_To_Roadways,1.050337


,0
Hillshade_9am * Slope,1.738763
Aspect * Hillshade_9am,0.446641
Horizontal_Distance_To_Roadways * Slope,0.416871
Soil_Type3,0.001775
Soil_Type4,0.001386
Soil_Type40,0.000433
Horizontal_Distance_To_Fire_Points * Horizontal_Distance_To_Roadways,0.000000
Soil_Type19,0.000000
Soil_Type13,0.000000
Soil_Type14,0.000000


# Print Scores/ Save Features

In [20]:

for name, log in logs.items():
    print('{}:'.format(name))
    print(log['score_original'])
    print(log['score_selected'])
    print()
    ft.save_features(log['feature_objs'],
                    location='features_{}.json'.format(name.lower()))

Squared_Root:
OrderedDict([('Random Forest', 0.7160429905352965), ('Extra Trees', 0.7493443978709718)])
OrderedDict([('Random Forest', 0.7179762216111908), ('Extra Trees', 0.7530111752772614)])

Abs_Diff:
OrderedDict([('Random Forest', 0.731411647452164), ('Extra Trees', 0.7580015267930983)])
OrderedDict([('Random Forest', 0.7271846924854919), ('Extra Trees', 0.7543877630360564)])

Mean_2_Cols:
OrderedDict([('Random Forest', 0.7305970043753932), ('Extra Trees', 0.7553048991680391)])
OrderedDict([('Random Forest', 0.7306659221194151), ('Extra Trees', 0.7624423034784023)])

Mean_3_Cols:
OrderedDict([('Random Forest', 0.7115686385388025), ('Extra Trees', 0.7405423649742353)])
OrderedDict([('Random Forest', 0.7137810748340673), ('Extra Trees', 0.7419383910710877)])

Div_2_Cols:
OrderedDict([('Random Forest', 0.7196072748863741), ('Extra Trees', 0.7435977182925364)])
OrderedDict([('Random Forest', 0.7263930219900617), ('Extra Trees', 0.7526330112459622)])

add_numeric:
OrderedDict([('Random

In [21]:
!ls .

__notebook_source__.ipynb  features_mean_3_cols.json
features_abs_diff.json	   features_multiply_numeric.json
features_add_numeric.json  features_squared_root.json
features_div_2_cols.json   features_subtract_numeric.json
features_mean_2_cols.json


In [22]:
# !head features_div_2_cols.json